In [10]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

import pytesseract
from PIL import Image, ImageDraw, ImageFont

import torch
from datasets import Dataset, Features, Sequence, ClassLabel, Value, Array2D
from transformers import LayoutLMTokenizer, LayoutLMForSequenceClassification, AdamW
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

current_path = os.path.dirname(os.path.abspath('__file__'))
root_path = os.path.dirname(current_path)
sys.path.append(root_path)

from IPython.display import display
import numpy as np
from PIL import Image
import tqdm as tqdm

# lib
from src.utils import apply_ocr,normalize_box,encode_testing_example

# nn
import torch
from transformers import LayoutLMTokenizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
dataset_path = "../data"
labels = [label for label in os.listdir(dataset_path)]
idx2label = {v: k for v, k in enumerate(labels)}
label2idx = {k: v for v, k in enumerate(labels)}
label2idx

{'email': 0, 'invoice': 1, 'resume': 2, 'scientific_publication': 3}

In [12]:
testing_features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'image_path': Value(dtype='string'),
    'words': Sequence(feature=Value(dtype='string')),
})

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LayoutLMForSequenceClassification.from_pretrained("saved_model")
model.to(device)

LayoutLMForSequenceClassification(
  (layoutlm): LayoutLMModel(
    (embeddings): LayoutLMEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 768)
      (y_position_embeddings): Embedding(1024, 768)
      (h_position_embeddings): Embedding(1024, 768)
      (w_position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LayoutLMEncoder(
      (layer): ModuleList(
        (0-11): 12 x LayoutLMLayer(
          (attention): LayoutLMAttention(
            (self): LayoutLMSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True

In [14]:
tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
query_df = pd.DataFrame(
    {'image_path': ['../testing_dataset/invoice1.png']}
)
query = Dataset.from_pandas(query_df)

query = query.map(apply_ocr)
query = query.map(lambda example: encode_testing_example(example), features=testing_features)

query.set_format(
    type='torch', columns=['input_ids', 'bbox', 'attention_mask', 'token_type_ids']
)
query = torch.utils.data.DataLoader(query, batch_size=1, shuffle=True)
batch = next(iter(query))

outputs = model(
    input_ids=batch["input_ids"].to(device), bbox=batch["bbox"].to(device), 
    attention_mask=batch["attention_mask"].to(device), 
    token_type_ids=batch["token_type_ids"].to(device)
)

preds = torch.softmax(outputs.logits, dim=1).tolist()[0]
pred_labels = {label:pred for label, pred in zip(label2idx.keys(), preds)}
print(pred_labels)

category_prediction = max(pred_labels, key=pred_labels.get)
print("document prediction : ",category_prediction)



{'email': 0.15688838064670563, 'invoice': 0.7861299514770508, 'resume': 0.029057005420327187, 'scientific_publication': 0.027924668043851852}
document prediction :  invoice
